In [1]:
import tensorflow as tf
import numpy as np
import cv2

In [25]:
def resnet50(
    inputs,
    classes
):
    x = tf.keras.layers.Conv2D(64, (7, 7), strides=(2, 2), padding='same')(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    
    x = tf.keras.layers.MaxPooling2D((3, 3), strides=(2, 2))(x)
    x = conv_block(x, 3, [64, 64, 256], strides=(1, 1))
    x = identity_block(x, 3, [64, 64, 256])
    x = identity_block(x, 3, [64, 64, 256])
    
    x = conv_block(x, 3, [128, 128, 512])
    x = identity_block(x, 3, [128, 128, 512])
    x = identity_block(x, 3, [128, 128, 512])
    x = identity_block(x, 3, [128, 128, 512])
    
    x = conv_block(x, 3, [256, 256, 1024])
    x = identity_block(x, 3, [256, 256, 1024])
    x = identity_block(x, 3, [256, 256, 1024])
    x = identity_block(x, 3, [256, 256, 1024])
    x = identity_block(x, 3, [256, 256, 1024])
    x = identity_block(x, 3, [256, 256, 1024])
    
    x = conv_block(x, 3, [512, 512, 2048])
    x = identity_block(x, 3, [512, 512, 2048])
    x = identity_block(x, 3, [512, 512, 2048])
    
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    pred = tf.keras.layers.Dense(classes, activation='softmax')(x)
    
    return pred

In [31]:
def identity_block(
    inputs,
    kernel_size,
    filters
):
    filters1, filters2, filters3 = filters
    x = tf.keras.layers.Conv2D(filters1, (1, 1), padding='same')(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    
    x = tf.keras.layers.Conv2D(filters2, (1, 1), padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    
    x = tf.keras.layers.Conv2D(filters3, kernel_size, padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    
    x = tf.keras.layers.add([x, inputs])
    x = tf.keras.layers.Activation('relu')(x)
    
    return x
    

In [24]:
def conv_block(
    inputs,
    kernel_size,
    filters,
    strides=(2,2)
):
    filters1, filters2, filters3 = filters
    
    x = tf.keras.layers.Conv2D(filters1, (1, 1), strides=strides, padding='same')(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    
    x = tf.keras.layers.Conv2D(filters2, (1, 1), padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    
    x = tf.keras.layers.Conv2D(filters3, (1, 1), padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    
    shortcut = tf.keras.layers.Conv2D(filters3, (1, 1), strides=strides, padding='same')(inputs)
    shortcut = tf.keras.layers.BatchNormalization()(shortcut)
    
    x = tf.keras.layers.add([x, shortcut])
    x = tf.keras.layers.Activation('relu')(x)
    
    return x

In [32]:
inputs = tf.keras.Input(shape=[32, 32, 3])
model = tf.keras.Model(inputs=inputs, outputs=resnet50(inputs, classes=10))
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_38 (Conv2D)              (None, 16, 16, 64)   9472        input_9[0][0]                    
__________________________________________________________________________________________________
batch_normalization_35 (BatchNo (None, 16, 16, 64)   256         conv2d_38[0][0]                  
__________________________________________________________________________________________________
activation_35 (Activation)      (None, 16, 16, 64)   0           batch_normalization_35[0][0]     
______________________________________________________________________________________________

In [34]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train = x_train / 255.0
y_train = tf.keras.utils.to_categorical(y_train, 10)

model.compile(
    loss = 'categorical_crossentropy',
    optimizer = 'adam',
    metrics = ['categorical_accuracy', 'Recall', 'AUC']
)

In [38]:
model.fit(x_train, y_train, epochs=1, batch_size=6)


Train on 50000 samples
  102/50000 [..............................] - ETA: 11:19:58 - loss: 3.7609 - categorical_accuracy: 0.1562 - Recall: 0.0833 - AUC: 0.5429

KeyboardInterrupt: 

In [42]:
model.save('my_model')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: my_model\assets


In [43]:
img = cv2.imread('ship.jpg', 1) /255.0
img = np.expand_dims(img, 0)

model = tf.keras.models.load_model('my_model')
pred = model.predict(img)
print(pred)

[[0.03218735 0.04736658 0.1596004  0.07240357 0.07138921 0.20864838
  0.2236265  0.04714107 0.08593269 0.05170422]]
